In [1]:
# %pip install pydantic-ai

In [3]:
import src.services.slack as srsl

await srsl.validate_bot_auth()

{'bot_user_id': 'U08KLT72BNZ',
 'bot_username': 'emmav22',
 'workspace': 'domo_user_group'}